# Auto Correct Using Probabilistic Model

In this notebook we will implement an Auto-correct Model that was first created by [Peter Norvig](https://en.wikipedia.org/wiki/Peter_Norvig) in 2007.<br>His [original article](https://norvig.com/spell-correct.html) may be a useful reference for this notebook.

### Simply an Auto-correct model is implementing these steps:
1. Identify Mispelled Words
2. Find words that are **N** edit distance away
- Edit Distances are operations performed on a word to change it to another word (check Below Cell)
3. Filter Candidates from generated word
4. Calculate Word Probability for Each Candidate

#### Edit Distance

In this notebook, I will implement models that correct words that are 1 and 2 edit distances away. 
- We say two words are n edit distance away from each other when we need n edits to change one word into another. 

An edit could consist of one of the following options: 

- Delete (remove a letter): ‘hat’ => ‘at, ha, ht’
- Switch (swap 2 adjacent letters): ‘eta’ => ‘eat, tea,...’
- Replace (change 1 letter to another): ‘jat’ => ‘hat, rat, cat, mat, ...’
- Insert (add a letter): ‘te’ => ‘the, ten, ate, ...’

#### Word Probability
A word probability could be simply calculated as:
$$P(c) = \frac{count(c)}{V} $$
Where $P(c) $ is the probability of correct word<br>
and $count(c) $ is the number of occurences of the word **c** in the vocabulary<br>
and $V $ is the total words in the vocabulary

With that being said, Let's start our notebook

## Data Preprocessing
As in any other machine learning task, the first thing you have to do is process your data set.<br>

we will generate our vocabulary using **Shakespere.txt** file<br>
1. We will read the file
2. Lowercase every word
3. Return List of words


In [6]:
import re

def process_data(file_name):
    """
    Input: 
        A file_name which is found in your current directory. You just have to read it in. 
    Output: 
        words: a list containing all the words in the corpus (text file you read) in lower case. 
    """
    words = []
    
    #Open the file, read its contents into a string variable
    with open(file_name) as file:
        string = file.read()
        
    # convert all letters to lower case
    lowered_string = string.lower()
    
    #Convert every word to lower case and return them in a list.
    texts = [sub_text for sub_text in lowered_string.split(" ")]

    pattern = "[a-zA-Z0-9]+"

    for text in texts:
        matches = re.findall(pattern, text)
        for match in matches:
            words.append(match)
                
    return words

In [7]:
word_l = process_data('./data/shakespeare.txt')
vocab = set(word_l)  # this will be your new vocabulary
print(f"The first ten words in the text are: \n{word_l[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the']
There are 6116 unique words in the vocabulary.


#### Now I'll Implement a get_count function that returns a dictionary
- The dictionary's keys are words
- The value for each word is the number of times that word appears in the corpus. 

In [8]:
def get_count(word_l):
    '''
    Input:
        word_l: a set of words representing the corpus. 
    Output:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    '''   
    word_count_dict = {}
    
    for word in word_l:
        word_count_dict[word] = word_count_dict.get(word,0) + 1
        
    return word_count_dict

In [11]:
word_count_dict = get_count(word_l)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'happy' is {word_count_dict.get('happy',0)}")

There are 6116 key values pairs
The count for the word 'happy' is 17


<HR></HR>


#### Now I'll Implement a get_probs function that returns a dictionary
Given the dictionary of word counts, I'll compute the probability that each word will appear if randomly selected from the corpus of words.<br>
$$P(c) = \frac{count(c)}{V} $$
Where $P(c) $ is the probability of correct word<br>
and $count(c) $ is the number of occurences of the word **c** in the vocabulary<br>
and $V $ is the total words in the vocabulary

#### get_probs function which gives you the probability that a word occurs in a sample. This returns a dictionary where
- keys are words 
- value for each word is its probability in the corpus of words.

In [14]:
import numpy as np

def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur. 
    '''
    probs = {}
    
    for word in list(word_count_dict.keys()):
        probs[word] = (word_count_dict[word])  / (np.sum(np.array(list(word_count_dict.values()))))
        
    return probs

In [15]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('happy') is {probs['happy']:.4f}")

Length of probs is 6116
P('happy') is 0.0003


<HR></HR>

# Word Operations

Now that you have computed $P(c)$ for all the words in the corpus, you will write a few functions that performs operations to words so that it can change mispelled words and return the right spellings of the words. I will implement four functions: 

* delete_letter : given a word, it returns all the possible strings that have **one character removed**. 
* switch_letter: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* replace_letter: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* insert_letter: given a word, it returns all the possible strings that have an **additional character inserted**. 

### Delete Operations
I will implement a delete_letter() function that, given a word, returns a list of strings with one character deleted. 

For example, given the word **nice**, it would return the set: {'ice', 'nce', 'nic', 'nie'}. 

Create a list of 'splits'. This is all the ways you can split a word into Left and Right: For example: <br>
nice is split into : **[('', 'nice'), ('n', 'ice'), ('ni', 'ce'), ('nic', 'e'), ('nice', '')]**

In [16]:
def delete_letter(word, verbose=False):
    '''
    Input:
        word: the string/word for which you will generate all possible words 
                in the vocabulary which have 1 missing character
    Output:
        delete_l: a list of all possible strings obtained by deleting 1 character from word
    '''
    
    delete_l = []
    split_l = []
    
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    delete_l = [L+R[1:] for L,R in split_l]
    
    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return  delete_l

In [19]:
delete_word_l = delete_letter(word="cans", verbose=True)
print()
print(f"Number of outputs of delete_letter('at') is {len(delete_letter('at'))}")

input word cans, 
split_l = [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's')], 
delete_l = ['ans', 'cns', 'cas', 'can']

Number of outputs of delete_letter('at') is 2


### Switch Operations
I will implement a switch_letter() function switches two letters in a word.<br> It takes in a word and returns a list of all the possible switches of two letters **that are adjacent to each other**. 

For example, given the word **'eta'**, it returns {'eat', 'tea'}, but does not return 'ate'.

In [20]:
def switch_letter(word, verbose=False):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent charater switched
    ''' 
    
    switch_l = []
    split_l = []
    
    
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    switch_l = [L[:-1]+R[0]+L[-1]+R[1:] for L,R in split_l if L and R]
    
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 
    
    return switch_l

In [22]:
switch_word_l = switch_letter(word="eta", verbose=True)
print()
print(f"Number of outputs of switch_letter('at') is {len(switch_letter('at'))}")

Input word = eta 
split_l = [('', 'eta'), ('e', 'ta'), ('et', 'a')] 
switch_l = ['tea', 'eat']

Number of outputs of switch_letter('at') is 1


### Replace Operations
I will implement a replace_letter() function that takes in a word and returns a list of strings with one replaced letter from the original word.

In [24]:
def replace_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    ''' 
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    
    replace_l = []
    split_l = []
    
    
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    replace_l = [L+N+R[1:] for L,R in split_l for N in letters if N != R[0]]
    replace_set = set(replace_l)
    
    
    # turn the set back into a list and sort it, for easier viewing
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

In [26]:
replace_l = replace_letter(word='can', verbose=True)
print()
print(f"Number of outputs of replace_letter('at') is {len(replace_letter('at'))}")

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']

Number of outputs of replace_letter('at') is 50


### Insert Operations
I will implement a insert_letter() function that takes in a word and returns a list with a letter inserted at every offset.

In [28]:
def insert_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    
    
    split_l = [(word[:i], word[i:]) for i in range(len(word)+1)]
    insert_l = [L+N+R for L,R in split_l for N in letters]
       
    
    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

In [30]:
insert_l = insert_letter('at', True)
print()
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word at 
split_l = [('', 'at'), ('a', 't'), ('at', '')] 
insert_l = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']

Number of strings output by insert_letter('at') is 78


<HR></HR>

## Preforming All Operations
Now that I have implemented the string manipulations, I will create two functions that, given a word, will return all the possible single and double edits on that word. These will be edit_one_letter() and edit_two_letters().

### One Letter Edit
I will implement the edit_one_letter() function to get all the possible edits that are one edit away from a word. The edits  consist of the replace, insert, delete, and optionally the switch operation.

In [31]:
def edit_one_letter(word, allow_switches = True):
    """
    Input:
        word: the string/word for which we will generate all possible wordsthat are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. Please return a set. and not a list.
    """
    
    edit_one_set = set()
    
    functions = [delete_letter, insert_letter, replace_letter]
    if allow_switches:
        functions.append(switch_letter)
        
    for func in functions:
        candidates = func(word)
        
        for candidate in candidates:
            edit_one_set.add(candidate)
    
    
    # return this as a set and not a list
    return set(edit_one_set)

In [32]:
tmp_word = "at"
tmp_edit_one_set = edit_one_letter(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"The type of the returned object should be a set {type(tmp_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")

input word at 
edit_one_l 
['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']

The type of the returned object should be a set <class 'set'>
Number of outputs from edit_one_letter('at') is 129


### Two Letter Edit
Now you can generalize this to implement to get two edits on a word. To do so, you would have to get all the possible edits on a single word and then for each modified word, you would have to modify it again.<br>

In [33]:
def edit_two_letters(word, allow_switches = True):
    '''
    Input:
        word: the input string/word 
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''
    
    edit_two_set = set()
    
    one_set_candidates = edit_one_letter(word, allow_switches)
    candidates_l = list(one_set_candidates)
    
    for candidate in candidates_l:
        second_degree_candidate = edit_one_letter(candidate, allow_switches)
        
        for cand in list(second_degree_candidate):
            edit_two_set.add(cand)
            
    for cand in candidates_l:
        edit_two_set.add(cand)
        
    
    # return this as a set instead of a list
    return set(edit_two_set)

In [34]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


<HR></HR>

# Suggest Words
Now I will use edit_two_letters function to get a set of all the possible 2 edits on your word. then using those strings to get the most probable word you meant to type a.k.a your typing suggestion.<br>
* If the word is in the vocabulary, suggest the word. 
* Otherwise, if there are suggestions from **edit_one_letter** that are in the vocabulary, use those. 
* Otherwise, if there are suggestions from **edit_two_letters** that are in the vocabulary, use those. 
* Otherwise, suggest the input word.*  
* The idea is that words generated from fewer edits are more likely than words with more edits.

Implementing **get_corrections** function, which returns a list of zero to n possible suggestion tuples of the form (word, probability_of_word).

Select the n best suggestions. There may be fewer than n.

In [35]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = []
    n_best = []
    
    #create suggestions    
    
    one_edits = list(edit_one_letter(word))
    two_edits = list(edit_two_letters(word))
    
    if word in vocab:
        suggestions.append(word)
    elif one_edits:
        for edit in one_edits:
            if edit in vocab:
                suggestions.append(edit)
    elif two_edits:
        for edit in two_edits:
            if edit in vocab:
                suggestions.append(edit)
    else :
        pass
    
    #determine probability of suggestions
    probabilities = [probs[suggest] for suggest in suggestions]
    
    #Get all your best words and return the most probable top n_suggested words as n_best
    sorted_idx = np.argsort(np.array(probabilities))[::-1][:n]
    suggestions = np.array(suggestions)[sorted_idx]
    probabilities = np.array(probabilities)[sorted_idx]
    n_best = list(zip(suggestions,probabilities))
        
    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best

In [49]:
my_word = 'kids' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True) 
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

print(f"data type of corrections {type(tmp_corrections)}")

entered word =  kids 
suggestions =  ['kiss' 'bids']
word 0: kiss, probability 0.000149
word 1: bids, probability 0.000075
data type of corrections <class 'list'>


<HR></HR>

# Conclusion
Since this Auto Correct is based on probablisitic model, it might not work properly.<br>
**Example**:<br>
`ands in your pants`, the correct word is **ants**, but since **and** has a higher probability the model will suggest **and**<br>
#### Future Work:
we can overcome this problem by gathering the context of the sentence using more complex models like RNN